In [1]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

path['project'] : D:\projects\skin-lesion-classification
path['images'] : D:\projects\skin-lesion-classification\images
path['models'] : D:\projects\skin-lesion-classification\models
path['expository'] : D:\projects\skin-lesion-classification\expository
path['literature'] : D:\projects\skin-lesion-classification\literature
path['notebooks'] : D:\projects\skin-lesion-classification\notebooks
path['presentation'] : D:\projects\skin-lesion-classification\presentation
path['scripts'] : D:\projects\skin-lesion-classification\scripts


In [2]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename
tvr: int = 3                        # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0                       # Random seed for parts of the process where randomness is called for.
keep_first: bool = False            # If False, then, for each lesion, we choose a random image to assign to our training set. 
stratified: bool = True             # If True, we stratify classes so that the proportions remain as stable as possible after train/val split. 
                                    # If False, the proportions will be roughly similar.
to_classify: list = ["mel",         # These are the lesion types we are interested in classifying. 
                     "bcc",         # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                     "akiec", 
                     "nv"]
sample_size: Union[None, dict] = {"mel": 2000,         
                                  "bcc": 2000, 
                                  "akiec": 2000, 
                                  "nv": 2000,
                                  "other" : 2000,} # Could also leave out "other" here, and include e.g. "df: 2000" if we wanted to.    

<details>
    <summary><b><i>Train test split explanation: click here to expand/collapse</i></b></summary>
    
We partition our dataset based on ```lesion_id```, **not** on ```image_id```: that way, every lesion will be represented in training or in validation, but not both.

For each classification task, we will train a model on
* **exactly one** image for every lesion in our training set;
* **all** images of every lesion in our training set.

In both cases, we will vaildate our model on 
* **exactly one** image for every lesion in our validation set;
* **all** images of every lesion in our validation set. 

**However**, we will make only one prediction per lesion (```lesion_id```) in our validation set, i.e. in the second case (validate on all images), if there are multiple images of a lesion in the validation set, we will combine the predictions for the multiple images into a single prediction for the lesion.

Accordingly, we proceed as follows: 
1. Randomly select (without replacement) a proportion of our $7470$ distinct ```lesion_id```s and label them with ```t``` (train).
2. Label the remaining ```lesion_id```s with ```v``` (validate).
3. For each ```lesion_id``` labeled with a ```t```:
    * Select an ```image_id``` and label it ```t1```.
    * Label all (if any) remaining ```image_id```s corresponding to this ```lesion_id``` with ```ta```.
4.  For each ```lesion_id``` labeled with a ```v```:
    * Select an ```image_id``` and label it ```v1```.
    * Label all (if any) remaining ```image_id```s corresponding to this ```lesion_id``` with ```va```.

In Step 1, the number of ```lesion_id```s randomly selected to be labeled ```t``` will be such that the ratio of ```t```s to ```v```s is as close as possible to a specified ratio (we default to $3$, i.e. $\approx75\%$ of lesions are represented in training). In Steps 3 and 4, the first substep can be done randomly (our default choice), or we can simply choose the "first" image in our table that corresponds to the lesion. 

The four train/val scenarios we consider are:
* ```t1v1```: train on precisely those images labeled ```t1``` and validate on precisely those labeled ```v1```.
* ```t1va```: train on precisely those images labeled ```t1``` and validate on precisely those labeled ```v1``` **or** ```va```.
* ```tav1```: train on precisely those images labeled ```t1``` **or** ```ta``` and validate on precisely those labeled ```v1```.
* ```tava```: train on precisely those images labeled ```t1``` **or** ```ta``` and validate on precisely those labeled ```v1``` ***or*** ```va```.

The mnemonic is ```t``` for training, ```v``` for validation, ```1``` for one-image-per-lesion, and ```a``` for all images.
</details>

In [3]:
# Create an instance of the process class with attribute values as above.
metadata = process(data_dir=data_dir,
                   csv_filename=csv_filename,
                   tvr=tvr,
                   seed=seed,
                   keep_first=keep_first,
                   stratified=stratified,
                   to_classify=to_classify,
                   sample_size=sample_size,)

Successfully loaded file 'D:\projects\skin-lesion-classification\images\metadata.csv'.
Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
Created self._label_dict (maps labels to indices).
Inserted 'label' column in dataframe, to the right of 'dx' column.
Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
Balancing classes in training set.
Re-combining balanced training set with validation set self._df_val_a so that everything is in one convenient dataframe: self._df_balanced.


In [4]:
divmod(2000,614)

(3, 158)

In [4]:
# Let's have a look at our metadata dataframe, which is now just an attribute of the metadata instance of the process class.
metadata.df.head()

,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set
0,HAM_0000118,2,ISIC_0027419,bkl,0,histo,80.0,male,scalp,ta
1,HAM_0000118,2,ISIC_0025030,bkl,0,histo,80.0,male,scalp,t1
2,HAM_0002730,2,ISIC_0026769,bkl,0,histo,80.0,male,scalp,va
3,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1
4,HAM_0001466,2,ISIC_0031633,bkl,0,histo,75.0,male,ear,va


In [5]:
for across in ["lesions", "images"]:
    for subset in ["all", "train", "val"]:
        process.dx_dist(metadata, subset = subset, across = across)

DISTRIBUTION OF LESIONS BY DIAGNOSIS: OVERALL


dx,nv,other,mel,bcc,akiec
freq,5403.00,898.00,614.00,327.00,228.00
%,72.33,12.02,8.22,4.38,3.05


Total lesions: 7470.

DISTRIBUTION OF LESIONS BY DIAGNOSIS: TRAIN


dx,nv,other,mel,bcc,akiec
freq,4052.00,673.00,460.00,245.00,171.00
%,72.34,12.02,8.21,4.37,3.05


Total lesions: 5601 (74.98% of all lesions).

DISTRIBUTION OF LESIONS BY DIAGNOSIS: VAL


...

Total images: 10015.

DISTRIBUTION OF IMAGES BY DIAGNOSIS: TRAIN


dx,nv,other,mel,bcc,akiec
freq,5007.00,1008.00,831.00,384.00,250.00
%,66.94,13.48,11.11,5.13,3.34


Total images: 7480 (74.69% of all images).

DISTRIBUTION OF IMAGES BY DIAGNOSIS: VAL


dx,nv,other,mel,bcc,akiec
freq,1698.00,348.00,282.00,130.00,77.00
%,66.98,13.73,11.12,5.13,3.04


Total images: 2535 (25.31% of all images).



In [4]:
# There are some implicit attributes of our process class:
metadata_hidden_attributes = metadata.get_hidden_attributes()
print(list(metadata_hidden_attributes.keys()))
# E.g.:
print(metadata_hidden_attributes["_label_codes"])

['_csv_file_path', '_label_dict', '_label_codes', '_num_labels', '_df_train1', '_df_train_a', '_df_val1', '_df_val_a', '_df_sample_batch', '_df_train_a_balanced']
{0: 'other', 1: 'akiec', 2: 'bcc', 3: 'mel', 4: 'nv'}


In [7]:
metadata._df_val_a

,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set
2,HAM_0002730,2,ISIC_0026769,bkl,0,histo,80.0,male,scalp,va
3,HAM_0002730,2,ISIC_0025661,bkl,0,histo,80.0,male,scalp,v1
4,HAM_0001466,2,ISIC_0031633,bkl,0,histo,75.0,male,ear,va
5,HAM_0001466,2,ISIC_0027850,bkl,0,histo,75.0,male,ear,v1
6,HAM_0002761,2,ISIC_0029176,bkl,0,histo,60.0,male,face,va
...,...,...,...,...,...,...,...,...,...,...
9993,HAM_0000545,2,ISIC_0026650,akiec,1,histo,70.0,male,face,va
9998,HAM_0004282,3,ISIC_0033358,akiec,1,histo,65.0,female,face,va
10000,HAM_0004282,3,ISIC_0033151,akiec,1,histo,65.0,female,face,v1
10003,HAM_0004592,2,ISIC_0029141,akiec,1,histo,60.0,female,face,va


In [7]:
# Now let's set values for the attributes of our resnet18 class (the model we will use with out processed data).
# One of the attributes has to do with image transformations.

import torchvision.transforms as transforms

transform = transforms.Compose([
transforms.CenterCrop((300, 300)),
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])    

In [8]:
import pandas as pd
from typing import List, Callable

df: pd.DataFrame = metadata.df                   # Background dataset for the model. metadata._df_sample_batch is a random selection of 64 rows of metadata.df. We use it for testing our code.
train_set: Union[pd.DataFrame, list, str] = "t1" # "t1" (one image per lesion in training set); ["t1", "ta"] (all images for each lesion in training set); can also specify another sub-dataframe of self.df.
val_set: Union[pd.DataFrame, list, str] = "v1"   # Similar to train_set above.
label_codes: dict = metadata._label_codes        # Correspondence between label codes like 0 and label words like 'other'.
data_dir: Path = path["images"]                  # Path to directory where images are stored.
model_dir: Path = path["models"]                 # Path to directory where models/model info/model results are stored.
transform: List[Callable] = transform            # Transform to be applied to images before feeding to ResNet-18
batch_size: int = 32                             # Mini-batch size: default 32.
epochs: int = 10                                 # Number of epochs (all layers unfrozen from the start): default 10.
base_learning_rate: float = 0.001                # Learning rate to start with: default 0.001. Using Adam optimizer.
filename_stem: str = "rn18mc"                    # For saving model and related files. train set and num epochs will be appended automatically. Default "rn18mc".
filename_suffix: str = "base"                    # Something descriptive and unique for future reference and to avoid over-writing other files. Default empty string "".

In [9]:
# # To reload custom module after editing, so we don't have to restart kernel and start over from the beginning every time.
# import importlib
# import multiclass_models
# # from multiclass_models import resnet18
# importlib.reload(multiclass_models);

In [10]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import resnet18

resnet18mc_test = resnet18(                                  # This instance is for testing the code.
    df=metadata.df.sample(n=16, random_state=metadata.seed), # Just a small number of rows for testing the code.
    train_set=train_set,
    val_set=val_set,
    label_codes=label_codes,
    data_dir=data_dir,
    model_dir=model_dir,
    transform=transform,
    batch_size=batch_size,
    epochs=1,                                                # Just a few epochs for testing the code.
    base_learning_rate=base_learning_rate,
    filename_stem=filename_stem,
    filename_suffix="test",                                  # Suffix "test" because we're just testing the code.
    Print = True,                                            # Print stuff while testing code to help find errors.
)

<details><summary>Click here to read about an error we found and (hopefully) corrected.</summary>
    
Below, we were catching ```RuntimeError: size mismatch (got input: [5], target: [1])```, before we incorporated one-hot encoding. This wasn't caught while testing on small batches of images, however: only after attempting to train on the entire training set on Google Colab.
    
Then, we caught a ```RuntimeError``` when we tested on dataset of size ```k``` (as in ```metadata.df.sample(n=k, random_state=metadata.seed)``` above) for these values of ```k```: ```1,12,13,14,15,16```. 

Sizes ```2,3,...,11,17,18,19,20,...,33``` were fine. Unfortunately, we were been thrown the same error when training on the full training set (on Google Colab) of size ~7,500, after a good 30 to 60 minutes. 

We thought it might be something to do with the size of the dataset module the batch size (```32```), but received no errors for sizes ```44,45```. Also, the number of epochs seemed irrelevant: changing number of epochs to ```1``` or ```2``` while keeping all else constant, had no effect as far as this error was concerned.

We noted that the error was thrown, at least in some cases (we did not re-check all sizes ```k``` again), not during the training loop, but during validation. We checked the validation sets ```resnet18mc_test._df_val``` that were being fed into the dataloader at this stage, and noticed that for all of the problematic ```k```, and none of the others, there was precisely one image in the validation set. We changed the line ```loss = criterion(outputs.squeeze(), labels)``` to ```loss = criterion(outputs, labels)``` in the validation loop, and found that the ```RuntimeError``` was no longer being thrown. We also made the same change to the training loop. We notice that, regardless of the use of the ```.squeeze()``` method, the ```outputs.shape``` seems to always be of the form ```torch.Size([m, 5])``` (when the code works), which is the right shape, making squeezing redundant.
</details>
<details><summary>Click here to see the error details.</summary>

```
    
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
Input In [83], in <cell line: 2>()
      1 # Train the model on the specified training data by calling the train method:
----> 2 resnet18mc_test.train()

File D:\projects\skin-lesion-classification\scripts\multiclass_models.py:195, in resnet18.train(self)
    193                     val_outputs = model(val_images)
    194 #                     val_loss = criterion(val_outputs.squeeze(), val_labels.long())
--> 195                     val_loss = criterion(val_outputs.squeeze(), val_labels)
    196                     if self.Print:
    197                         print(f"outputs.shape: {outputs.shape}")

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1511, in Module._wrapped_call_impl(self, *args, **kwargs)
   1509     return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1510 else:
-> 1511     return self._call_impl(*args, **kwargs)

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1520, in Module._call_impl(self, *args, **kwargs)
   1515 # If we don't have any hooks, we want to skip the rest of the logic in
   1516 # this function, and just call forward.
   1517 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1518         or _global_backward_pre_hooks or _global_backward_hooks
   1519         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1520     return forward_call(*args, **kwargs)
   1522 try:
   1523     result = None

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:1179, in CrossEntropyLoss.forward(self, input, target)
   1178 def forward(self, input: Tensor, target: Tensor) -> Tensor:
-> 1179     return F.cross_entropy(input, target, weight=self.weight,
   1180                            ignore_index=self.ignore_index, reduction=self.reduction,
   1181                            label_smoothing=self.label_smoothing)

File ~\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:3059, in cross_entropy(input, target, weight, size_average, ignore_index, reduce, reduction, label_smoothing)
   3057 if size_average is not None or reduce is not None:
   3058     reduction = _Reduction.legacy_get_string(size_average, reduce)
-> 3059 return torch._C._nn.cross_entropy_loss(input, target, weight, _Reduction.get_enum(reduction), ignore_index, label_smoothing)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported
```
</details>

In [11]:
print("Training set".upper())
display(resnet18mc_test._df_train)
print("Validation set".upper())
display(resnet18mc_test._df_val)

TRAINING SET


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set
6144,HAM_0002695,1,ISIC_0028664,nv,4,follow_up,45.0,male,back,t1
4658,HAM_0000370,1,ISIC_0025998,nv,4,follow_up,70.0,male,trunk,t1
2767,HAM_0005536,1,ISIC_0026798,bcc,3,histo,45.0,male,lower extremity,t1
1359,HAM_0005084,2,ISIC_0027261,mel,1,histo,75.0,male,ear,t1
4097,HAM_0007158,1,ISIC_0027206,nv,4,follow_up,50.0,female,lower extremity,t1
4503,HAM_0003274,1,ISIC_0031348,nv,4,follow_up,40.0,female,upper extremity,t1
5529,HAM_0003796,1,ISIC_0030696,nv,4,follow_up,50.0,male,trunk,t1
5508,HAM_0005574,1,ISIC_0027823,nv,4,follow_up,35.0,female,trunk,t1
1087,HAM_0000215,1,ISIC_0025484,bkl,0,consensus,70.0,female,lower extremity,t1
1328,HAM_0002838,1,ISIC_0030818,mel,1,histo,65.0,female,lower extremity,t1


VALIDATION SET


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set
1070,HAM_0003691,1,ISIC_0027428,bkl,0,consensus,75.0,female,back,v1


In [12]:
# Train the model on the specified training data by calling the train method:
resnet18mc_test.train()

image_id, label, ohe-label: ISIC_0031348, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0027261, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0024505, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0025998, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0027206, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0028664, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0025484, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0026798, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0030818, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0030696, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0027823, 4, tensor([0., 0., 0., 0., 1.])
outputs.shape: torch.Size([11, 5])
loss: 1.7371655702590942
Validating...
image_id, label, ohe-label: ISIC_0027428, 0, tensor([1., 0., 0., 0., 0.])
outputs.shape: torch.Size([11, 5])
val

In [13]:
# Let's look at the training and validation loss for each epoch:
resnet18mc_test.epoch_losses

{'train_loss': array([1.73716557]), 'val_loss': array([1.24101424])}

In [14]:
# The model will be saved as a .pth file in the directory given by model_dir attribute.
# Sans .pth extension, the filename is
resnet18mc_test._filename

'rn18mc_t1_1e_test_00'

<details><summary>Click here to read about an error encountered when using Google Colab's GPU for inference.</summary>
    
To fix, added line ```images = images.to(device)``` in definition of ```inference``` function:
    
```
with torch.no_grad():
    for images, labels, image_ids in dataloader:
        # Send input tensor to device
        images = images.to(device)
        # Make predictions using the model
        outputs = model(images)
        # Apply softmax to get probabilities                
        probabilities = softmax(outputs)
```

```
    
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-14-4e2245e9b2a7> in <cell line: 3>()
      1 # We can feed our entire dataframe through the trained model to obtain predictions for all lesions/images.
      2 # Data can be loaded from a pre-saved .pth file if it is not still in memory.
----> 3 inference_df = resnet18mc_test.inference()
      4 display(inference_df)

8 frames
/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts/multiclass_models.py in inference(self, df_infer, filename, Print)
    258             for images, labels, image_ids in dataloader:
    259                 # Make predictions using the model
--> 260                 outputs = model(images)
    261                 # Apply softmax to get probabilities
    262                 probabilities = softmax(outputs)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _wrapped_call_impl(self, *args, **kwargs)
   1509             return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1510         else:
-> 1511             return self._call_impl(*args, **kwargs)
   1512 
   1513     def _call_impl(self, *args, **kwargs):

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _call_impl(self, *args, **kwargs)
   1518                 or _global_backward_pre_hooks or _global_backward_hooks
   1519                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1520             return forward_call(*args, **kwargs)
   1521 
   1522         try:

/usr/local/lib/python3.10/dist-packages/torchvision/models/resnet.py in forward(self, x)
    283 
    284     def forward(self, x: Tensor) -> Tensor:
--> 285         return self._forward_impl(x)
    286 
    287 

/usr/local/lib/python3.10/dist-packages/torchvision/models/resnet.py in _forward_impl(self, x)
    266     def _forward_impl(self, x: Tensor) -> Tensor:
    267         # See note [TorchScript super()]
--> 268         x = self.conv1(x)
    269         x = self.bn1(x)
    270         x = self.relu(x)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _wrapped_call_impl(self, *args, **kwargs)
   1509             return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]
   1510         else:
-> 1511             return self._call_impl(*args, **kwargs)
   1512 
   1513     def _call_impl(self, *args, **kwargs):

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py in _call_impl(self, *args, **kwargs)
   1518                 or _global_backward_pre_hooks or _global_backward_hooks
   1519                 or _global_forward_hooks or _global_forward_pre_hooks):
-> 1520             return forward_call(*args, **kwargs)
   1521 
   1522         try:

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py in forward(self, input)
    458 
    459     def forward(self, input: Tensor) -> Tensor:
--> 460         return self._conv_forward(input, self.weight, self.bias)
    461 
    462 class Conv3d(_ConvNd):

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py in _conv_forward(self, input, weight, bias)
    454                             weight, bias, self.stride,
    455                             _pair(0), self.dilation, self.groups)
--> 456         return F.conv2d(input, weight, bias, self.stride,
    457                         self.padding, self.dilation, self.groups)
    458 

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor
    
```
</details>

<details><summary>Click here to read about a subsequent error encountered when using Google Colab's GPU for inference.</summary> The above fix did not break the inference process when a CPU was in use, however on the GPU... ```TypeError``` below was thrown. 
    
Fix: in definition of inference function, we went from   
    
```
# Apply softmax to get probabilities                
                probabilities = softmax(outputs)
        
                series_dict = { }
                series_dict["image_id"] = pd.Series(image_ids)

                for idx, label in enumerate(self.label_codes.values()):
                    series_dict["prob_" + label] = pd.Series(probabilities[:,idx])    
```   

to this:    
    
    
```
# Apply softmax to get probabilities                
                probabilities = softmax(outputs)
                
                # Move probabilities to CPU before converting to NumPy array
                probabilities_cpu = probabilities.cpu().numpy()
        
                series_dict = { }
                series_dict["image_id"] = pd.Series(image_ids)

                for idx, label in enumerate(self.label_codes.values()):
                    series_dict["prob_" + label] = pd.Series(probabilities_cpu[:,idx])    
```    
    
    
    
TypeError                                 Traceback (most recent call last)
<ipython-input-16-4e2245e9b2a7> in <cell line: 3>()
      1 # We can feed our entire dataframe through the trained model to obtain predictions for all lesions/images.
      2 # Data can be loaded from a pre-saved .pth file if it is not still in memory.
----> 3 inference_df = resnet18mc_test.inference()
      4 display(inference_df)

3 frames
/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts/multiclass_models.py in inference(self, df_infer, filename, Print)
    268 
    269                 for idx, label in enumerate(self.label_codes.values()):
--> 270                     series_dict["prob_" + label] = pd.Series(probabilities[:,idx])
    271 
    272                 batch_df = pd.DataFrame(series_dict)

/usr/local/lib/python3.10/dist-packages/pandas/core/series.py in __init__(self, data, index, dtype, name, copy, fastpath)
    468                     data = data.copy()
    469             else:
--> 470                 data = sanitize_array(data, index, dtype, copy)
    471 
    472                 manager = get_option("mode.data_manager")

/usr/local/lib/python3.10/dist-packages/pandas/core/construction.py in sanitize_array(data, index, dtype, copy, raise_cast_failure, allow_2d)
    614         if hasattr(data, "__array__"):
    615             # e.g. dask array GH#38645
--> 616             data = np.array(data, copy=copy)
    617         else:
    618             data = list(data)

/usr/local/lib/python3.10/dist-packages/torch/_tensor.py in __array__(self, dtype)
   1060             return handle_torch_function(Tensor.__array__, (self,), self, dtype=dtype)
   1061         if dtype is None:
-> 1062             return self.numpy()
   1063         else:
   1064             return self.numpy().astype(dtype, copy=False)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.    
    
```

In [15]:
# We can feed our entire dataframe through the trained model to obtain predictions for all lesions/images.
# Data can be loaded from a pre-saved .pth file if it is not still in memory.
resnet18mc_test.inference(save=True);

image_id, label, ohe-label: ISIC_0028664, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0025998, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0032817, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0026577, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0026798, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0027261, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0027206, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0031348, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0025752, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0030696, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0027823, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0027428, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0025484, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_00298

In [16]:
display(resnet18mc_test._inference_df.head())

,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_akiec,prob_bcc,prob_nv
0,HAM_0002695,1,ISIC_0028664,nv,4,follow_up,45.0,male,back,t1,0.115207,0.035088,0.069254,0.095782,0.684669
1,HAM_0000370,1,ISIC_0025998,nv,4,follow_up,70.0,male,trunk,t1,0.146937,0.04739,0.086763,0.039126,0.679784
2,HAM_0006372,3,ISIC_0032817,nv,4,histo,35.0,male,back,ta,0.086053,0.076374,0.108117,0.042361,0.687095
3,HAM_0006835,2,ISIC_0026577,nv,4,histo,75.0,male,chest,ta,0.126639,0.149507,0.259437,0.072006,0.392411
4,HAM_0005536,1,ISIC_0026798,bcc,3,histo,45.0,male,lower extremity,t1,0.001576,0.000564,0.003631,0.991735,0.002494


In [17]:
# Or we can make predictions for individual lesions/images:
display(resnet18mc_test.prediction("HAM_0005084"))
display(resnet18mc_test.prediction("ISIC_0032817"))

image_id, label, ohe-label: ISIC_0027261, 1, tensor([0., 1., 0., 0., 0.])


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_akiec,prob_bcc,prob_nv
0,HAM_0005084,2,ISIC_0027261,mel,1,histo,75.0,male,ear,t1,0.056206,0.737477,0.053019,0.07288,0.080418


image_id, label, ohe-label: ISIC_0032817, 4, tensor([0., 0., 0., 0., 1.])


,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_akiec,prob_bcc,prob_nv
0,HAM_0006372,3,ISIC_0032817,nv,4,histo,35.0,male,back,ta,0.086053,0.076374,0.108117,0.042361,0.687095


In [18]:
resnet18mc_test.state_dict

OrderedDict([('conv1.weight',
              tensor([[[[-1.1419e-02, -7.1356e-03, -2.8098e-03,  ...,  5.5615e-02,
                          1.6083e-02, -1.3694e-02],
                        [ 1.0083e-02,  8.5276e-03, -1.1093e-01,  ..., -2.7224e-01,
                         -1.3007e-01,  2.7424e-03],
                        [-7.9433e-03,  5.8089e-02,  2.9448e-01,  ...,  5.1872e-01,
                          2.5532e-01,  6.2573e-02],
                        ...,
                        [-2.8535e-02,  1.5045e-02,  7.1595e-02,  ..., -3.3385e-01,
                         -4.2158e-01, -2.5881e-01],
                        [ 2.9613e-02,  3.9960e-02,  6.1850e-02,  ...,  4.1284e-01,
                          3.9259e-01,  1.6506e-01],
                        [-1.4736e-02, -4.6746e-03, -2.5084e-02,  ..., -1.5170e-01,
                         -8.3230e-02, -6.7828e-03]],
              
                       [[-1.0397e-02, -2.5619e-02, -3.3641e-02,  ...,  3.3521e-02,
                          1.6622

In [19]:
# Let's check that the code works if our state dictionary is no longer in memory, so that we have to load it from a .pth file.
resnet18mc_test.state_dict = None
display(resnet18mc_test.inference(filename = resnet18mc_test._filename))

image_id, label, ohe-label: ISIC_0028664, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0025998, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0032817, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0026577, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0026798, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0027261, 1, tensor([0., 1., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0027206, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0031348, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0025752, 3, tensor([0., 0., 0., 1., 0.])
image_id, label, ohe-label: ISIC_0030696, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0027823, 4, tensor([0., 0., 0., 0., 1.])
image_id, label, ohe-label: ISIC_0027428, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_0025484, 0, tensor([1., 0., 0., 0., 0.])
image_id, label, ohe-label: ISIC_00298

,lesion_id,num_images,image_id,dx,label,dx_type,age,sex,localization,set,prob_other,prob_mel,prob_akiec,prob_bcc,prob_nv
0,HAM_0002695,1,ISIC_0028664,nv,4,follow_up,45.0,male,back,t1,0.115207,0.035088,0.069254,0.095782,0.684669
1,HAM_0000370,1,ISIC_0025998,nv,4,follow_up,70.0,male,trunk,t1,0.146937,0.04739,0.086763,0.039126,0.679784
2,HAM_0006372,3,ISIC_0032817,nv,4,histo,35.0,male,back,ta,0.086053,0.076374,0.108117,0.042361,0.687095
3,HAM_0006835,2,ISIC_0026577,nv,4,histo,75.0,male,chest,ta,0.126639,0.149507,0.259437,0.072006,0.392411
4,HAM_0005536,1,ISIC_0026798,bcc,3,histo,45.0,male,lower extremity,t1,0.001576,0.000564,0.003631,0.991735,0.002494
5,HAM_0005084,2,ISIC_0027261,mel,1,histo,75.0,male,ear,t1,0.056206,0.737477,0.053019,0.07288,0.080418
6,HAM_0007158,1,ISIC_0027206,nv,4,follow_up,50.0,female,lower extremity,t1,0.065724,0.074954,0.112205,0.029411,0.717707
7,HAM_0003274,1,ISIC_0031348,nv,4,follow_up,40.0,female,upper extremity,t1,0.054394,0.063993,0.04986,0.025166,0.806587
8,HAM_0005435,3,ISIC_0025752,bcc,3,histo,65.0,male,back,ta,0.048059,0.082101,0.116378,0.063831,0.689632
9,HAM_0003796,1,ISIC_0030696,nv,4,follow_up,50.0,male,trunk,t1,0.094317,0.071804,0.066598,0.050858,0.716423


In [20]:
resnet18mc_base = resnet18(                                  
    df=metadata.df, 
    train_set=train_set,
    val_set=val_set,
    label_codes=label_codes,
    data_dir=data_dir,
    model_dir=model_dir,
    transform=transform,
    batch_size=batch_size,
    epochs=epochs,                                                
    base_learning_rate=base_learning_rate,
    filename_stem=filename_stem,
    filename_suffix="base",                                  
)

In [21]:
infer = pd.read_csv(resnet18mc_base.model_dir.joinpath("rn18mc_ta_10e_base_inference.csv"))

In [22]:
resnet18mc_base._df_infer = pd.merge(resnet18mc_base.df, infer, on="image_id", how="left")

In [23]:
infer = resnet18mc_base._df_infer.copy()

In [24]:
inverse_label_codes = {value: key for key, value in resnet18mc_base.label_codes.items()}

def get_argmax(row):
    probabilities = row[['prob_other', 'prob_mel', 'prob_akiec', 'prob_nv', 'prob_bcc']].astype(float)
    max_column = probabilities.idxmax()
    dx = max_column.split('_')[1]  # Split the string and return the second part (after 'prob_')    
    return inverse_label_codes[dx]


In [25]:
infer['pred'] = infer.apply(get_argmax, axis=1)

In [26]:
infer_val = infer[infer["set"] == "v1"]

In [110]:
from evaluation import custom_confusion

confusion = custom_confusion(infer_val['label'], infer_val['pred'], label_codes)

confusion.keys()

dict_keys(['labels_by_predictions', 'prob_label_given_prediction', 'prob_prediction_given_label', 'label_given_prediction_merged', 'prediction_given_label_merged'])

In [118]:
display(confusion["labels_by_predictions"])
print("="*40 + "\nProb(label | prediction = column header)\n" + "="*40)
display(confusion["label_given_prediction_merged"])
print("="*35 + "\nProb(prediction | label = row name)\n" + "="*35)
display(confusion["prediction_given_label_merged"])

pred,other,mel,akiec,bcc,nv,All
label,,,,,,
other,81,26,12,9,97,225
mel,9,68,8,5,64,154
akiec,7,8,17,18,7,57
bcc,9,1,4,62,6,82
nv,34,97,27,34,1159,1351
All,140,200,68,128,1333,1869


Prob(label | prediction = column header)


pred,other,mel,akiec,bcc,nv,All
label,,,,,,
other,81 (↓57.86%),26 (↓13.0%),12 (↓17.65%),9 (↓7.03%),97 (↓7.28%),225 (↓12.04%)
mel,9 (↓6.43%),68 (↓34.0%),8 (↓11.76%),5 (↓3.91%),64 (↓4.8%),154 (↓8.24%)
akiec,7 (↓5.0%),8 (↓4.0%),17 (↓25.0%),18 (↓14.06%),7 (↓0.53%),57 (↓3.05%)
bcc,9 (↓6.43%),1 (↓0.5%),4 (↓5.88%),62 (↓48.44%),6 (↓0.45%),82 (↓4.39%)
nv,34 (↓24.29%),97 (↓48.5%),27 (↓39.71%),34 (↓26.56%),1159 (↓86.95%),1351 (↓72.28%)
All,140 (↓100.0%),200 (↓100.0%),68 (↓100.0%),128 (↓100.0%),1333 (↓100.0%),1869 (↓100.0%)


Prob(prediction | label = row name)


pred,other,mel,akiec,bcc,nv,All
label,,,,,,
other,81 (→36.0%),26 (→11.56%),12 (→5.33%),9 (→4.0%),97 (→43.11%),225 (→100%)
mel,9 (→5.84%),68 (→44.16%),8 (→5.19%),5 (→3.25%),64 (→41.56%),154 (→100%)
akiec,7 (→12.28%),8 (→14.04%),17 (→29.82%),18 (→31.58%),7 (→12.28%),57 (→100%)
bcc,9 (→10.98%),1 (→1.22%),4 (→4.88%),62 (→75.61%),6 (→7.32%),82 (→100%)
nv,34 (→2.52%),97 (→7.18%),27 (→2.0%),34 (→2.52%),1159 (→85.79%),1351 (→100%)
All,140 (→7.49%),200 (→10.7%),68 (→3.64%),128 (→6.85%),1333 (→71.32%),1869 (→100%)
